In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Conv2D, AveragePooling2D,Input,BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense,Concatenate
from keras.utils import to_categorical
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras import backend as K
import numpy as np

Using TensorFlow backend.


In [3]:
#设置图像数据
img_width, img_height = 50, 50

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
    bn_axis = 1
else:
    input_shape = (img_width, img_height, 3)
    bn_axis = 3

x = Input (shape = input_shape)    

#生成一个可分支模型
#分支一
branch1_out = Conv2D(16, (1, 1),
    padding="same",
    use_bias=False)(x)
branch1_out = BatchNormalization(axis=bn_axis)(branch1_out)
branch1_out = Activation('relu')(branch1_out)
    
    
#分支二   
branch2_out = Conv2D(
         16, (1, 1),
         padding="same",
         use_bias=False)(x)
branch2_out = BatchNormalization(axis=bn_axis)(branch2_out)
branch2_out = Activation('relu')(branch2_out)    
branch2_out = Conv2D(
         48, (3, 3),
         padding="same",
         use_bias=False)(branch2_out)
branch2_out = BatchNormalization(axis=bn_axis)(branch2_out)
branch2_out = Activation('relu')(branch2_out) 


#分支三
branch3_out = Conv2D(
       16, (1, 1),
        padding="same",
        use_bias=False)(x)
branch3_out = BatchNormalization(axis=bn_axis)(branch3_out)
branch3_out = Activation('relu')(branch3_out)
branch3_out = Conv2D(
         24, (5, 5),
         padding="same",
         use_bias=False)(branch3_out)
branch3_out = BatchNormalization(axis=bn_axis)(branch3_out)
branch3_out = Activation('relu')(branch3_out) 

#branch 4
branch4_out = AveragePooling2D(
        pool_size=(3, 3),strides=(1, 1), padding='same', data_format=K.image_data_format())(x)
branch4_out = Conv2D(
         16, (1, 1),
         padding="same",
         use_bias=False)(branch4_out)
branch4_out = BatchNormalization(axis=bn_axis)(branch4_out)
branch4_out = Activation('relu')(branch4_out) 

#连接层
out = Concatenate(axis=bn_axis)([branch1_out, branch2_out, branch3_out, branch4_out])
out = Conv2D(
         16, (1, 1),
         padding="same",
         use_bias=False)(out)
#全链接层
out = Flatten()(out)
out = Dense(48, activation='relu')(out)
#输出
out = Dense(1, activation='sigmoid')(out)

model = Model(x, out)

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [4]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 50, 50, 3)    0                                            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 50, 50, 16)   48          input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 50, 50, 16)   48          input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_8 (BatchNor (None, 50, 50, 16)   64          conv2d_9[0][0]                   
__________________________________________________________________________________________________
batch_norm

In [5]:
#训练加载路径
train_data_dir = r'E:/pycharm/dogs-vs-cats/train'
validation_data_dir = r'E:/pycharm/dogs-vs-cats/validation'
nb_train_samples = 10835
nb_validation_samples = 4000
epochs = 1
batch_size = 5

#训练数据大小变化
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)


test_datagen = ImageDataGenerator(rescale=1. / 255)
#生成训练数据
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')



Found 10835 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


In [6]:
#训练
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/1
2167/2167 [==============================] - 264s 122ms/step - loss: 8.0516 - acc: 0.4950 - val_loss: 8.0550 - val_acc: 0.5003


In [9]:
import cv2
img = cv2.resize(cv2.imread(r'E:/pycharm/dogs-vs-cats/test/4.jpg'), (img_width, img_height)).astype(np.float32)

x = img_to_array(img)

x = np.expand_dims(x, axis=0)



score = model.predict(x)


print(score)

[[0.]]
